In [9]:
###-Repositorio estadistica informatica pregrado UNALM-### 

#Trabajo Final → AVANCE.
#1. Relación de repositorios con sus respectivos url
#2. Automatización de la descarga de la información de un repositorio.
#3. Estructuración de la información descargada en formato csv

url = 'http://repositorio.lamolina.edu.pe/handle/20.500.12996/26/recent-submissions'

import urllib.request, urllib.error
from bs4 import BeautifulSoup
import re as re 

#url de la siguiente pagina
def next_page(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page)
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('\?offset.*','',url)
    n_url = url + url_diff
    return n_url

#url de todas las paginas 
URL = []
url1 = url

def all_pages(url,URL):
    
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page)
    last = soup.find('li',{'class':'next pull-right disabled'})
    URL.append(url)
    if last == None: 
        url = next_page(url)
        all_pages(url,URL)
    else: 
        return URL 
    return URL 
all_pages(url1,URL)

###Almacenar links de las tesis 

#tittles =[]
handles = [] 

for urls in URL: 
    page = urllib.request.urlopen(urls)
    soup = BeautifulSoup(page)
    for lines in soup.find_all('h4'):
        lines = lines.decode().strip()
        link = re.findall(r'(?<=20.500.12996/).+?(?=\")',lines) #handle unico a cada tesis 
        handles.append(link)
        #tittle = re.findall(r'(?<=>)\w.+?(?=<)',lines) #Titulo de la tesis 
        #tittles.append(tittle)

## Links completos 

links=[]
def get_links(handles,links):
    handle_url = re.sub('(?<=20.500.12996/).+','',url)
    for i in range(len(handles)): 
        links.append(handle_url+handles[i][0])
    return links

get_links(handles,links)    

links_full = [] 
for link in links: 
    links_full.append(link+'?show=full')

##test 
page1 = urllib.request.urlopen(links_full[0])
soup1 = BeautifulSoup(page1)
full_t=''

for line in soup1.find_all('tr'):
    full_t = line.decode().strip()+'\n'+full_t

grantor = re.findall(r'(?<=>thesis\.degree\.grantor</td><td class=\"word-break\">).+?(?=</td>)',full_t)
print(grantor)

#La información descargada debe estar estructurada de la forma siguiente:
#- Institución (universidad) //test//
#- Título de la tesis
#- Nombre del tesista
#- Grado (‘para obtener el grado de:’ … bachiller, ingeniero, licenciado, maestro,
#doctor, etc)
#- Nombre del asesor(es)
#- Resumen
#- Año


['Universidad Nacional Agraria La Molina. Facultad de Economía y Planificación']
